In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from google.colab import files

In [2]:
market_url="https://www.ft.com/apple-inc?page="
pages=range(1,40)

In [3]:
from types import NoneType
news_tables={}
parsed_heads=[]
parsed_dates=[]
for page in pages:
  url= market_url + str(page)

  req=Request(url=url, headers={'user-agent':'my-app'})
  response=urlopen(req)

  html=BeautifulSoup(response,'html')
  news_table=html.find('ul',{'class':'o-teaser-collection__list js-stream-list'})
  news_tables[page]=news_table

  list_item=news_tables[page].findAll('li',{'class':'o-teaser-collection__item o-grid-row'})
  list_headings=news_tables[page].findAll('a',{'class':'js-teaser-standfirst-link'})
  list_dates=news_tables[page].findAll('div',{'class':'stream-card__date'})
  for row in enumerate(list_headings):
    #if type(row[1].a)==NoneType or row[1].a.text == '':
      #continue
    headline=row[1].text.strip()
    parsed_heads.append(headline)
  for row in enumerate(list_dates):
    time=row[1].time.text
    parsed_dates.append(time)

In [4]:
import pandas as pd
df = pd.DataFrame(list(zip(parsed_dates, parsed_heads)),
               columns =['Date', 'News'])

In [5]:
df

,Date,News
0,"Friday, 2 February, 2024",Tech giant’s shares lower even as earnings bea...
1,"Friday, 2 February, 2024",Groups behind Adele and Phoebe Bridgers among ...
2,"Thursday, 1 February, 2024",‘Mixed reality’ headset released in US on Frid...
3,"Tuesday, 30 January, 2024",And China’s halfhearted stimulus
4,"Thursday, 25 January, 2024",Schiller’s criticism comes as company announce...
...,...,...
943,"Friday, 4 January, 2019",Move aimed at heading off another legal challe...
944,"Friday, 4 January, 2019","Chinese consumers are loyal to Apple, but not ..."
945,"Thursday, 3 January, 2019",The reputation of big tech faltered as scandal...
946,"Thursday, 3 January, 2019",Qualcomm accuses US tech group of disregarding...


In [6]:
df['Date']=pd.to_datetime(df['Date'])
df=df.set_index('Date', inplace=False)
df = df[~df.index.duplicated()]
df = df.resample('D').asfreq()
df['News'].fillna(method='ffill', inplace=True)
df['ds']=df.index
df=df.reset_index(drop=True)
df.head()

,News,ds
0,The reputation of big tech faltered as scandal...,2019-01-03
1,Investors seek safe haven as analysts warn of ...,2019-01-04
2,Investors seek safe haven as analysts warn of ...,2019-01-05
3,Investors seek safe haven as analysts warn of ...,2019-01-06
4,Company flags fewer people upgrading iPhones i...,2019-01-07


In [7]:
df

,News,ds
0,The reputation of big tech faltered as scandal...,2019-01-03
1,Investors seek safe haven as analysts warn of ...,2019-01-04
2,Investors seek safe haven as analysts warn of ...,2019-01-05
3,Investors seek safe haven as analysts warn of ...,2019-01-06
4,Company flags fewer people upgrading iPhones i...,2019-01-07
...,...,...
1852,Schiller’s criticism comes as company announce...,2024-01-29
1853,And China’s halfhearted stimulus,2024-01-30
1854,And China’s halfhearted stimulus,2024-01-31
1855,‘Mixed reality’ headset released in US on Frid...,2024-02-01


In [8]:
df.shape

(1857, 2)

In [9]:
df.head(100)

,News,ds
0,The reputation of big tech faltered as scandal...,2019-01-03
1,Investors seek safe haven as analysts warn of ...,2019-01-04
2,Investors seek safe haven as analysts warn of ...,2019-01-05
3,Investors seek safe haven as analysts warn of ...,2019-01-06
4,Company flags fewer people upgrading iPhones i...,2019-01-07
...,...,...
95,Latest move to expand consumer lending pitches...,2019-04-08
96,"Walmart, Tesco and Apple among groups that did...",2019-04-09
97,Court rules one of chipmaker’s key patents as ...,2019-04-10
98,Court rules one of chipmaker’s key patents as ...,2019-04-11


In [10]:
df.tail()

,News,ds
1852,Schiller’s criticism comes as company announce...,2024-01-29
1853,And China’s halfhearted stimulus,2024-01-30
1854,And China’s halfhearted stimulus,2024-01-31
1855,‘Mixed reality’ headset released in US on Frid...,2024-02-01
1856,Tech giant’s shares lower even as earnings bea...,2024-02-02


In [13]:
df = df[df['ds'] <= '2024-01-01']

In [14]:
df

,News,ds
0,The reputation of big tech faltered as scandal...,2019-01-03
1,Investors seek safe haven as analysts warn of ...,2019-01-04
2,Investors seek safe haven as analysts warn of ...,2019-01-05
3,Investors seek safe haven as analysts warn of ...,2019-01-06
4,Company flags fewer people upgrading iPhones i...,2019-01-07
...,...,...
1820,There is a noticeable gap between tech company...,2023-12-28
1821,Contract manufacturer taking over iPhone assem...,2023-12-29
1822,Contract manufacturer taking over iPhone assem...,2023-12-30
1823,Contract manufacturer taking over iPhone assem...,2023-12-31


In [15]:
df.to_csv('Apple_sentiment.csv', index=False)

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [17]:
sia=SentimentIntensityAnalyzer()
compounder=lambda title: sia.polarity_scores(title)['compound']
df['market_compound']=df['News'].apply(compounder)

<ipython-input-17-9a8e30ac98eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['market_compound']=df['News'].apply(compounder)


In [18]:
def encode_sentiment(sentiment_value):
    if sentiment_value > 0.1:
        return 1
    elif sentiment_value < -0.1:
        return -1
    else:
        return 0

# Apply the encoding function to create a new 'sentiment_encoded' column
df['sentiment_encoded'] = df['market_compound'].apply(encode_sentiment)
df.head()

<ipython-input-18-1e61ce8ce4b3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_encoded'] = df['market_compound'].apply(encode_sentiment)


,News,ds,market_compound,sentiment_encoded
0,The reputation of big tech faltered as scandal...,2019-01-03,-0.4939,-1
1,Investors seek safe haven as analysts warn of ...,2019-01-04,-0.1779,-1
2,Investors seek safe haven as analysts warn of ...,2019-01-05,-0.1779,-1
3,Investors seek safe haven as analysts warn of ...,2019-01-06,-0.1779,-1
4,Company flags fewer people upgrading iPhones i...,2019-01-07,-0.6124,-1


In [20]:
df.head(10)

,News,ds,market_compound,sentiment_encoded
0,The reputation of big tech faltered as scandal...,2019-01-03,-0.4939,-1
1,Investors seek safe haven as analysts warn of ...,2019-01-04,-0.1779,-1
2,Investors seek safe haven as analysts warn of ...,2019-01-05,-0.1779,-1
3,Investors seek safe haven as analysts warn of ...,2019-01-06,-0.1779,-1
4,Company flags fewer people upgrading iPhones i...,2019-01-07,-0.6124,-1
5,Apple chief executive explains cut to Q1 guidance,2019-01-08,-0.2732,-1
6,Companies dependent on iPhone sales lead techn...,2019-01-09,-0.2960,-1
7,The iPhone maker saw its shares slide 9% after...,2019-01-10,-0.0516,0
8,Suppliers and luxury groups hit by fears of sl...,2019-01-11,-0.4215,-1
9,Suppliers and luxury groups hit by fears of sl...,2019-01-12,-0.4215,-1


In [21]:
df.shape

(1825, 4)

In [25]:
df.rename(columns={'ds': 'Date'}, inplace=True)

<ipython-input-25-04e36d89f130>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'ds': 'Date'}, inplace=True)


In [26]:
df.head()

,News,Date,market_compound,sentiment_encoded
0,The reputation of big tech faltered as scandal...,2019-01-03,-0.4939,-1
1,Investors seek safe haven as analysts warn of ...,2019-01-04,-0.1779,-1
2,Investors seek safe haven as analysts warn of ...,2019-01-05,-0.1779,-1
3,Investors seek safe haven as analysts warn of ...,2019-01-06,-0.1779,-1
4,Company flags fewer people upgrading iPhones i...,2019-01-07,-0.6124,-1


In [29]:
df['Date'] = pd.to_datetime(df['Date'])

<ipython-input-29-b9c6864122be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])


In [22]:
hist=pd.read_csv('filled_stock_data_AAPL.csv')

In [23]:
hist.shape

(1823, 7)

In [24]:
hist.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,38.722500,39.712502,38.557499,39.480000,37.893337,148158800
1,2019-01-03,35.994999,36.430000,35.500000,35.547501,34.118874,365248800
2,2019-01-04,36.132500,37.137501,35.950001,37.064999,35.575386,234428400
3,2019-01-05,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
4,2019-01-06,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
5,2019-01-07,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
6,2019-01-08,37.389999,37.955002,37.130001,37.687500,36.172871,164101200
7,2019-01-09,37.822498,38.632500,37.407501,38.327499,36.787151,180396400
8,2019-01-10,38.125000,38.492500,37.715000,38.450001,36.904724,143122800
9,2019-01-11,38.220001,38.424999,37.877499,38.072498,36.542397,108092800


In [31]:
hist['Date'] = pd.to_datetime(hist['Date'])

In [32]:
merged_df = pd.merge(df, hist, on='Date', how='inner')

print(merged_df)

                                                   News       Date  \
0     The reputation of big tech faltered as scandal... 2019-01-03   
1     Investors seek safe haven as analysts warn of ... 2019-01-04   
2     Investors seek safe haven as analysts warn of ... 2019-01-05   
3     Investors seek safe haven as analysts warn of ... 2019-01-06   
4     Company flags fewer people upgrading iPhones i... 2019-01-07   
...                                                 ...        ...   
1817  As mobile computing invades markets such as he... 2023-12-25   
1818  Tech company had appealed against earlier ruli... 2023-12-26   
1819  Biden administration had previously declined t... 2023-12-27   
1820  There is a noticeable gap between tech company... 2023-12-28   
1821  Contract manufacturer taking over iPhone assem... 2023-12-29   

      market_compound  sentiment_encoded        Open        High         Low  \
0             -0.4939                 -1   35.994999   36.430000   35.500000   

In [33]:
merged_df.head(10)

,News,Date,market_compound,sentiment_encoded,Open,High,Low,Close,Adj Close,Volume
0,The reputation of big tech faltered as scandal...,2019-01-03,-0.4939,-1,35.994999,36.430000,35.500000,35.547501,34.118874,365248800
1,Investors seek safe haven as analysts warn of ...,2019-01-04,-0.1779,-1,36.132500,37.137501,35.950001,37.064999,35.575386,234428400
2,Investors seek safe haven as analysts warn of ...,2019-01-05,-0.1779,-1,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
3,Investors seek safe haven as analysts warn of ...,2019-01-06,-0.1779,-1,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
4,Company flags fewer people upgrading iPhones i...,2019-01-07,-0.6124,-1,37.174999,37.207500,36.474998,36.982498,35.496204,219111200
5,Apple chief executive explains cut to Q1 guidance,2019-01-08,-0.2732,-1,37.389999,37.955002,37.130001,37.687500,36.172871,164101200
6,Companies dependent on iPhone sales lead techn...,2019-01-09,-0.2960,-1,37.822498,38.632500,37.407501,38.327499,36.787151,180396400
7,The iPhone maker saw its shares slide 9% after...,2019-01-10,-0.0516,0,38.125000,38.492500,37.715000,38.450001,36.904724,143122800
8,Suppliers and luxury groups hit by fears of sl...,2019-01-11,-0.4215,-1,38.220001,38.424999,37.877499,38.072498,36.542397,108092800
9,Suppliers and luxury groups hit by fears of sl...,2019-01-12,-0.4215,-1,37.712502,37.817501,37.305000,37.500000,35.992905,129756800


In [35]:
merged_df.to_csv('combined.csv', index=False)